In [1]:
# Importing all the necessary libraries we will be needing to do the Ananlysis


import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

Exracting the content in a wiki page that has 'List of US Cities by population' in to a text file

In [44]:
link = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
page = requests.get(link) 
soup = BeautifulSoup(page.text)

Finding the table that has the data i.e. list of all cities with their population, Square Area, Location

In [5]:
table = soup.find_all('table')[4]

Extracting the table from the webpage into a data frame by specifying the column names

In [6]:

table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=["Rank", "City", "State", "del1", "del2", "del3", "Sq.Area", "del5", "population density in Sq Mi", "Population density in Km2", "Location"])
df.head()

,Rank,City,State,del1,del2,del3,Sq.Area,del5,population density in Sq Mi,Population density in Km2,Location
0,1,New York[d],New York,"8,336,817","8,175,133",+1.98%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,"3,979,576","3,792,621",+4.93%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,"2,693,976","2,695,598",−0.06%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston[3],Texas,"2,320,268","2,100,263",+10.48%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,5,Phoenix,Arizona,"1,680,992","1,445,632",+16.28%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...


Finding the radius of each city with the help of Sq.Area, this step involves in preprocessing of the the column Sq.Area (changing its data type to float) then finding its square root

Drop unnecessary columns like Rank, del1, del2 ... del5, population density in Sq Mi from the data frame we extracted from the webpage

In [8]:
df.drop(columns = ["Rank", "del1", "del2", "del3", "del5", "Sq.Area", "population density in Sq Mi"], inplace = True)
df

,City,State,Population density in Km2,Location
0,New York[d],New York,"10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,Los Angeles,California,"3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,Chicago,Illinois,"4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,Houston[3],Texas,"1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,Phoenix,Arizona,"1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...
5,Philadelphia[e],Pennsylvania,"4,511/km2",40°00′34″N 75°08′00″W﻿ / ﻿40.0094°N 75.1333°W﻿...
6,San Antonio,Texas,"1,250/km2",29°28′21″N 98°31′30″W﻿ / ﻿29.4724°N 98.5251°W﻿...
7,San Diego,California,"1,670/km2",32°48′55″N 117°08′06″W﻿ / ﻿32.8153°N 117.1350°...
8,Dallas,Texas,"1,493/km2",32°47′36″N 96°45′59″W﻿ / ﻿32.7933°N 96.7665°W﻿...
9,San Jose,California,"2,231/km2",37°17′48″N 121°49′08″W﻿ / ﻿37.2967°N 121.8189°...



Splitting the cooridnates to Latitudes and Longitudes for each city

In [9]:
df["Location"]= df["Location"].str.split("/", n = 2, expand = True)[1]
df.head()

,City,State,Population density in Km2,Location
0,New York[d],New York,"10,933/km2",﻿40.6635°N 73.9387°W﻿
1,Los Angeles,California,"3,276/km2",﻿34.0194°N 118.4108°W﻿
2,Chicago,Illinois,"4,600/km2",﻿41.8376°N 87.6818°W﻿
3,Houston[3],Texas,"1,395/km2",﻿29.7866°N 95.3909°W﻿
4,Phoenix,Arizona,"1,200/km2",﻿33.5722°N 112.0901°W﻿


In [10]:
new = df["Location"].str.split(" ", n = 0, expand = False)
k = df.copy(deep = True)

In [15]:
df.head()

,City,State,Population density in Km2,Location
0,New York[d],New York,"10,933/km2",﻿40.6635°N 73.9387°W﻿
1,Los Angeles,California,"3,276/km2",﻿34.0194°N 118.4108°W﻿
2,Chicago,Illinois,"4,600/km2",﻿41.8376°N 87.6818°W﻿
3,Houston[3],Texas,"1,395/km2",﻿29.7866°N 95.3909°W﻿
4,Phoenix,Arizona,"1,200/km2",﻿33.5722°N 112.0901°W﻿


Getting the per capita income state wise for USA

In [46]:
link1 = 'https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income'
page1 = requests.get(link1) 
soup1 = BeautifulSoup(page1.text)

In [65]:
table = soup1.find_all('table')[1]
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df_state.head()

,RankCountry or Country-equivalent
0,This article is part of a series on
1,Topics\nHousehold\nPersonal\nAffluence\nSocial...
2,Lists by income\nStates (by equality (Gini))\n...
3,United States portal
4,vte


Define Four Square Credentials and version

In [68]:

CLIENT_ID = 'AN3SMQTKLWGT3NKKC3YDJI11QUXFOGDCP4LXFGPPZVVGGN21' # your Foursquare ID
CLIENT_SECRET = 'A3OP23GQ5C1GQXBLD3QE4TLONNW0XXHWS23BWEI4V5UNMNGL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 20
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AN3SMQTKLWGT3NKKC3YDJI11QUXFOGDCP4LXFGPPZVVGGN21
CLIENT_SECRET:A3OP23GQ5C1GQXBLD3QE4TLONNW0XXHWS23BWEI4V5UNMNGL


In [69]:

def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng,radius in zip(names, latitudes, longitudes,radius):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
       # print(results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [85]:
# Final coordinates of the place where we will be setting up an arcade is the one that has maximum weight for, in the above data frame
lat1 = 40.720102
long1 = -74.048121

In [86]:
# create map of the venues that we have using latitude and longitudes
final_map = folium.Map(location=[lat1, long1], zoom_start=15) # generate map centred around Jersey city


# add prefered location in the City as a green circle mark
folium.features.CircleMarker(
    [lat1, long1],
    radius=50,
    popup='Gaming arcade can be installed within this circle',
    fill=True,
    color='green',
    fill_color='green',
    fill_opacity=0.6
    ).add_to(final_map)
final_map

This place is between the Groove Street and the Grand Street